## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bredasdorp,ZA,-34.5322,20.0403,64.27,68,31,16.11,scattered clouds
1,1,Westport,US,41.1415,-73.3579,45.81,96,100,1.01,light rain
2,2,Bereda,ES,43.2686,-7.5406,52.00,89,49,3.29,scattered clouds
3,3,Nikolskoye,RU,59.7035,30.7861,6.69,100,98,1.14,overcast clouds
4,4,Cherskiy,RU,68.7500,161.3000,-28.52,99,6,1.30,clear sky


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature (in Fahrenheit) you would like for your trip? "))
max_temp = float(input("What is the maximum temperature (in Fahrenheit) you would like for your trip? "))

What is the minimum temperature (in Fahrenheit) you would like for your trip? 60
What is the maximum temperature (in Fahrenheit) you would like for your trip? 70


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bredasdorp,ZA,-34.5322,20.0403,64.27,68,31,16.11,scattered clouds
6,6,Hermanus,ZA,-34.4187,19.2345,64.90,78,2,8.75,clear sky
16,16,Kalabo,ZM,-14.9700,22.6814,67.64,94,100,3.47,moderate rain
38,38,Nishihara,JP,26.1842,127.7558,61.88,94,100,5.75,mist
41,41,Luderitz,NaN,-26.6481,15.1594,68.85,71,2,16.53,clear sky
53,53,Constitucion,CL,-35.3333,-72.4167,67.86,66,10,7.78,clear sky
54,54,Cape Town,ZA,-33.9258,18.4232,67.08,64,40,1.99,scattered clouds
62,62,Broken Hill,AU,-31.9500,141.4333,60.51,51,0,12.66,clear sky
64,64,Coquimbo,CL,-29.9533,-71.3436,68.76,69,14,8.46,few clouds
70,70,Magaria,NE,12.9983,8.9099,67.68,13,0,10.67,clear sky


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                78
City                   78
Country                76
Lat                    78
Lng                    78
Max Temp               78
Humidity               78
Cloudiness             78
Wind Speed             78
Current Description    78
dtype: int64

In [9]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [10]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Bredasdorp,ZA,64.27,scattered clouds,-34.5322,20.0403,
6,Hermanus,ZA,64.90,clear sky,-34.4187,19.2345,
16,Kalabo,ZM,67.64,moderate rain,-14.9700,22.6814,
38,Nishihara,JP,61.88,mist,26.1842,127.7558,
53,Constitucion,CL,67.86,clear sky,-35.3333,-72.4167,
54,Cape Town,ZA,67.08,scattered clouds,-33.9258,18.4232,
62,Broken Hill,AU,60.51,clear sky,-31.9500,141.4333,
64,Coquimbo,CL,68.76,few clouds,-29.9533,-71.3436,
70,Magaria,NE,67.68,clear sky,12.9983,8.9099,
72,Si Sa Ket,TH,69.28,scattered clouds,14.8333,104.3333,


In [45]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city. In addition, grab the lat and long coorinates for the hotel.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        hotel_df.loc[index, "Lat"] = hotels["results"][0]["geometry"]["location"]["lat"]
        hotel_df.loc[index, "Lng"] = hotels["results"][0]["geometry"]["location"]["lng"]       
    except (IndexError):
        print("Hotel not found... skipping.")    
        

Hotel not found... skipping.


In [46]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Bredasdorp,ZA,64.27,scattered clouds,-34.532971,20.035894,Bredasdorp Country Manor
6,Hermanus,ZA,64.90,clear sky,-34.421390,19.236641,Misty Waves Boutique Hotel
16,Kalabo,ZM,67.64,moderate rain,-14.989748,22.681215,Sikakubete Guest House Kalabo Town Council
38,Nishihara,JP,61.88,mist,26.202921,127.760063,Minshuku Agaihama
53,Constitucion,CL,67.86,clear sky,-35.333303,-72.418517,Hotel Boutique Las Azucenas
54,Cape Town,ZA,67.08,scattered clouds,-33.899615,18.412039,"Radisson Blu Hotel Waterfront, Cape Town"
62,Broken Hill,AU,60.51,clear sky,-31.957779,141.465727,Royal Exchange Hotel
64,Coquimbo,CL,68.76,few clouds,-29.985274,-71.359010,Hotel San Juan
70,Magaria,NE,67.68,clear sky,13.003848,8.904166,Chez Honorable Chapiou
72,Si Sa Ket,TH,69.28,scattered clouds,14.827495,104.342556,ร้านค้า อัญชัน


In [47]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df=hotel_df.dropna()

In [48]:
# 8a. Create the output File (CSV)
output_hotel_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_hotel_file, index_label="City_ID")

In [49]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [51]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# 11b. Display the figure
fig = gmaps.figure(center=(25.0, 10.0), zoom_level=1.5)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))